## Data Analysis and AI ML Model Predictor

 ##### Welcome! This is my project called -> Predicting Ultra-Marathon Performance, Data Driven Approach using Specific Marathon Event trends and Athlete metadata

 Made by Bryan Lee Santoso

In [ ]:
# Importing all necessary libraries
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import plotly.express as px
import streamlit as st
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Importing the dataset
# The dataset is assumed to be in the same directory as this script
df = pd.read_csv("TWO_CENTURIES_OF_UM_RACES.csv")

In [ ]:
# Since it contains 7,461,226 ultra-marathon race records from 1,641,168 athletes, we will just sample a bit
df_sample = df.sample(frac=0.10, random_state=42)  # 10% sample

In [ ]:
# Fixing the column types
# Convert dates and extract features
df_sample['Event dates'] = pd.to_datetime(df_sample['Event dates'], errors='coerce')
df_sample['Event_year'] = df_sample['Event dates'].dt.year
df_sample['Event_season'] = df_sample['Event dates'].dt.quarter  # 1=Winter, 2=Spring, etc.


In [ ]:
#Clean up null values for data validity
df_sample.isnull().sum()
df_sample = df_sample.dropna()

In [ ]:
df_sample.reset_index(drop=True)


In [ ]:
df_sample['Athlete performance'] = df_sample['Athlete performance'].str.split(' ').str.get(0)

In [ ]:
df_sample = df_sample.drop(['Athlete club'], axis = 1)

In [ ]:
df_sample.reset_index(drop=True)


In [ ]:
# Check for missing values in each column
print(df_sample.isna().sum())

In [ ]:
# Standardizing the athlete performance to total seconds
df_sample['Athlete performance'] = pd.to_timedelta(df_sample['Athlete performance'].str.replace(' h', ''), errors='coerce').dt.total_seconds()

In [ ]:
df_sample = df_sample[df_sample['Event distance/length'].isin(['50km', '100km'])]

In [ ]:
# Extract numeric distance from the string and convert to float
df_sample['Event distance/length'] = df_sample['Event distance/length'].str.replace('km', '').astype(float)

In [ ]:
df_sample['Athlete average speed'] = pd.to_numeric(df_sample['Athlete average speed'], errors='coerce')

In [ ]:
df_sample.isnull().sum()
df_sample = df_sample.dropna(subset = ['Athlete average speed'])

In [ ]:
# Calculate athlete's age at the event
df_sample['Athlete age'] = df_sample['Year of event'] - df_sample['Athlete year of birth']

In [ ]:
# Extract country from event name
df_sample['Event country'] = df_sample['Event name'].str.extract(r'\((.*?)\)')

In [ ]:
# Check for duplicates
print(df_sample.duplicated().sum())

# Drop duplicates
df_sample = df_sample.drop_duplicates()

In [ ]:
# Final checking of the data types
print(df_sample.dtypes)
df_sample.shape

In [ ]:
# Just for representation, since we have two categories of races, 50km and 100km, we can view the statistics of each categories by seperating them first
df_50km = df_sample[df_sample['Event distance/length'] == 50]
df_100km = df_sample[df_sample['Event distance/length'] == 100]

In [ ]:
sns.boxplot(x='Event distance/length', y='Athlete performance', data=df_50km)
plt.show()
sns.boxplot(x='Event distance/length', y='Athlete performance', data=df_100km)
plt.show()

df_50km['Athlete performance'].describe()

In [ ]:
# As there are outliers, we want to clean up the data first before continuing, we will clean up each category first, and then merge the dataset back together in order to prepare the final dataset for the model. We are focusing primarily on athlete performance

# First we want to clean the 50km dataset first
# Calculate Q1 (25th percentile) and Q3 (75th percentile)

Q1_50 = df_50km['Athlete performance'].quantile(0.25)
Q3_50 = df_50km['Athlete performance'].quantile(0.75)

# Calculate the IQR
IQR_50 = Q3_50 - Q1_50

# Define lower and upper bounds
lower_bound_50 = Q1_50 - 1.5 * IQR_50
upper_bound_50 = Q3_50 + 1.5 * IQR_50

print(f"Lower Bound (50km): {lower_bound_50}, Upper Bound (50km): {upper_bound_50}")



In [ ]:
# Next we want to calculate the statitics for the 100km races
# Calculate Q1 (25th percentile) and Q3 (75th percentile)

Q1_100 = df_100km['Athlete performance'].quantile(0.25)
Q3_100 = df_100km['Athlete performance'].quantile(0.75)

# Calculate the IQR
IQR_100 = Q3_100 - Q1_100

# Define lower and upper bounds
lower_bound_100 = Q1_100 - 1.5 * IQR_100
upper_bound_100 = Q3_100 + 1.5 * IQR_100

print(f"Lower Bound (100km): {lower_bound_100}, Upper Bound (100km): {upper_bound_100}")



In [ ]:
# If we remove the outliers based on the IQR rule, it will significantly reduce the dataset, thus we will take Winsorize the outliers , capping the outliers at the 2nd and 98th percentiles
# Cap outliers at the 2nd and 98th percentiles
lower_cap_50 = df_50km['Athlete performance'].quantile(0.02)
upper_cap_50 = df_50km['Athlete performance'].quantile(0.98)

df_50km['Athlete performance'] = df_50km['Athlete performance'].clip(lower=lower_cap_50, upper=upper_cap_50)

In [ ]:
sns.boxplot(x='Event distance/length', y='Athlete performance', data=df_50km)
plt.show()
df_50km.head(10)

In [ ]:
# If we remove the outliers based on the IQR rule, it will significantly reduce the dataset, thus we will take Winsorize the outliers , capping the outliers at the 2nd and 98th percentiles
# Cap outliers at the 2nd and 98th percentiles
lower_cap_100 = df_100km['Athlete performance'].quantile(0.02)
upper_cap_100 = df_100km['Athlete performance'].quantile(0.98)

df_100km['Athlete performance'] = df_100km['Athlete performance'].clip(lower=lower_cap_100, upper=upper_cap_100)

In [ ]:
sns.boxplot(x='Event distance/length', y='Athlete performance', data=df_100km)
plt.show()
df_100km.head(10)

In [ ]:
# After data cleaning, we will concatenate the two datasets back together
final_df = pd.concat([df_50km, df_100km], axis=0, ignore_index=True)

In [ ]:
# We will drop unrelated columns that are not needed for the model
final_df = final_df.drop(['Year of event', 'Event dates', 'Athlete year of birth', 'Athlete average speed', 'Athlete ID'], axis=1)

In [ ]:
final_df.head(10)

In [ ]:
final_df = final_df.drop(['Event number of finishers', 'Athlete country', 'Athlete country', 'Athlete age category','Event_year'], axis = 1)

In [ ]:
final_df.head(10)
final_df['Event name'] = final_df['Event name'].str.split('(').str.get(0)

In [ ]:
final_df.head(10)

In [ ]:
# Beginning the ML model Traning
# Idea -> By inputting the Athlete age and gender, Event name, Event distance, and Event season, we are able to predict the Athlete's performance
# Constrains, models are only predicting in races Kilometer race cetegories

In [ ]:
# First, we do feature engineering through target encoding
# Calculate the average performance for each event
event_performance_mean = final_df.groupby('Event name')['Athlete performance'].mean()

In [ ]:
# Replace the event name with its mean performance
final_df['Event name (encoded)'] = final_df['Event name'].map(event_performance_mean)
final_df.head(10)
final_df = final_df.drop(['Event country'], axis=1)


In [ ]:
# # To prevent information leakage, split the data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Split the dataset into training and test sets
X = final_df.drop(columns=['Athlete performance'])
y = final_df['Athlete performance']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Calculate target encoding based on training data
event_performance_mean = X_train.merge(y_train, left_index=True, right_index=True).groupby('Event name')['Athlete performance'].mean()

X_train['Event name (encoded)'] = X_train['Event name'].map(event_performance_mean)
X_test['Event name (encoded)'] = X_test['Event name'].map(event_performance_mean)

# Handle unseen events in the test set (fallback to global mean if the input event was not included in the original dataset) 
global_mean = y_train.mean()
X_test['Event name (encoded)'] = X_test['Event name (encoded)'].fillna(global_mean)

X_train = X_train.drop(columns=['Event name'])
X_test = X_test.drop(columns=['Event name'])

label_encoder = LabelEncoder()
X_train['Athlete gender'] = label_encoder.fit_transform(X_train['Athlete gender'])
X_test['Athlete gender'] = label_encoder.transform(X_test['Athlete gender'])

final_df = final_df.drop(columns=['Event name'])


In [ ]:
# Since the values of athlete performance are continuous data, then we will use a simple Random Forest Regressor

# Initialize the model
model = RandomForestRegressor(random_state=42)
# Train the model on the training set
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R² Score: {r2}")

In [ ]:
feature_importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': model.feature_importances_
}).sort_values(by='Importance', ascending=False)

# Plot feature importances
plt.barh(feature_importances['Feature'], feature_importances['Importance'])
plt.xlabel('Importance')
plt.ylabel('Feature')
plt.title('Feature Importance')
plt.show()

In [ ]:
# import joblib
# joblib.dump(model, 'athlete_performance_model.pkl')
# loaded_model = joblib.load('athlete_performance_model.pkl')

In [ ]:
# Example user input
user_input = {
    'Event name': 'Flatland Marathon',
    'Event distance/length': 50,
    'Event season': 2,
    'Athlete age': 30,
    'Athlete gender': 'M'
}

# Preprocess the user input
user_input_encoded = user_input.copy()
user_input_encoded['Event name (encoded)'] = event_performance_mean.get(
    user_input['Event name'], global_mean 
)
user_input_encoded.pop('Event name')
user_input_encoded['Athlete gender'] = label_encoder.transform([user_input['Athlete gender']])[0]
user_input_df = pd.DataFrame([user_input_encoded])
user_input_df = user_input_df.reindex(columns=X_train.columns, fill_value=0)


user_prediction = model.predict(user_input_df)
print(f"Predicted Athlete Performance: {user_prediction[0]}")